In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
train= pd.read_csv('Train_Data.csv')
test=pd.read_csv('Test_Data.csv')
train = train[train["age_group"].notnull()]
train.head()

,SEQN,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,73564.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,73568.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult
2,73576.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,Adult
3,73577.0,1.0,2.0,28.9,104.0,NaN,84.0,16.15,Adult
4,73580.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,Adult


In [35]:
for column in ["SEQN","BMXBMI","LBXGLU","LBXGLT","LBXIN"]:
    mean = train[column].mean()
    std = train[column].std()
    train["z_score"] = (train[column] - mean) / std
    train.loc[train["z_score"].abs() > 3, column] = np.nan
    train.drop(columns=["z_score"], inplace=True)

In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

le = LabelEncoder()
train['age_group']=le.fit_transform(train['age_group'])

imputer = SimpleImputer(strategy='median')
train[train.columns] = imputer.fit_transform(train)
train.isnull().sum()

SEQN         0
RIAGENDR     0
PAQ605       0
BMXBMI       0
LBXGLU       0
DIQ010       0
LBXGLT       0
LBXIN        0
age_group    0
dtype: int64

In [37]:
X= train.drop(columns=["age_group"])
Y=train["age_group"]
x_train,x_test,y_train,y_test = train_test_split(X,Y ,test_size=0.3, random_state=42) 

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

LR = LogisticRegression(class_weight='balanced')
LR.fit(X_train_scaled, y_train)
y_pred_lr = LR.predict(X_test_scaled) 
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

         0.0       0.92      0.75      0.83       501
         1.0       0.29      0.60      0.39        85

    accuracy                           0.73       586
   macro avg       0.60      0.68      0.61       586
weighted avg       0.83      0.73      0.76       586



In [39]:
for column in ["SEQN","BMXBMI","LBXGLU","LBXGLT","LBXIN"]:
    mean = test[column].mean()
    std = test[column].std()
    test["z_score"] = (test[column] - mean) / std
    test.loc[test["z_score"].abs() > 3, column] = np.nan
    test.drop(columns=["z_score"], inplace=True)

In [40]:
test[test.columns] = imputer.fit_transform(test)
test.isnull().sum()

SEQN        0
RIAGENDR    0
PAQ605      0
BMXBMI      0
LBXGLU      0
DIQ010      0
LBXGLT      0
LBXIN       0
dtype: int64

In [41]:
test_scaled= scaler.transform(test)
ytest_pred = LR.predict(test_scaled)
df = pd.DataFrame(ytest_pred, columns=['age_group'])
df.age_group.unique()

array([0., 1.])

In [43]:
df.to_csv("predicted_submission.csv",index=False)
df

,age_group
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0
...,...
307,0.0
308,0.0
309,1.0
310,0.0
